In [1]:
%env CUDA_VISIBLE_DEVICES=8

from random import randint

import torch
torch.set_float32_matmul_precision('high')
import torch.nn.functional as F
import triton
import triton.language as tl

env: CUDA_VISIBLE_DEVICES=8


## Triton Kernels

In [2]:
@triton.jit
def rtn_fp4(x):
    x_abs = tl.abs(x)
    x_sign = tl.where(
        x > 0,
        1,
        -1,
    )
    x_fp4_abs = tl.where(
        x_abs >= 5,
        6,
        tl.where(
            x_abs >= 3.5,
            4,
            tl.where(
                x_abs >= 2.5,
                3,
                tl.where(
                    x_abs >= 1.75,
                    2,
                    tl.where(
                        x_abs >= 1.25,
                        1.5,
                        tl.where(
                            x_abs >= 0.75,
                            1,
                            tl.where(
                                x_abs >= 0.25,
                                0.5,
                                0.0,
                            )
                        )
                    )
                )
            )
        )
    )
    return x_fp4_abs * x_sign


@triton.jit
def get_scales(x, amax, val_max, scales_max):
    s_dec = tl.where(
        amax == 0.0,
        1.0,
        amax / scales_max / val_max,
    )
    
    s_dec_b = tl.max(tl.abs(x), axis=-1, keep_dims=True) / val_max
    s_dec_b_e4m3 = (s_dec_b / s_dec).to(tl.float8e4nv).to(tl.float32)
    s_dec_b_e4m3 = tl.where(
        s_dec_b_e4m3 == 0,
        1.0,
        s_dec_b_e4m3,
    )
    return s_dec_b_e4m3, s_dec


@triton.jit
def get_alt_scales(x, val_max, s_dec):    
    s_dec_b = tl.max(tl.abs(x), axis=-1, keep_dims=True) / val_max
    s_dec_b_e4m3 = (s_dec_b * (6/4) / s_dec).to(tl.float8e4nv).to(tl.float32)
    s_dec_b_e4m3 = tl.where(
        s_dec_b_e4m3 == 0,
        1.0,
        s_dec_b_e4m3,
    )
    return s_dec_b_e4m3


@triton.autotune(
    configs=[
        triton.Config({"BLOCK_SIZE": 128 * 32}),
        triton.Config({"BLOCK_SIZE": 256 * 32}),
        triton.Config({"BLOCK_SIZE": 512 * 32}),
    ],
    key=[],
)
@triton.jit
def rtn_1x16s_fp4_kernel(
    x_ptr,
    amax_ptr,
    output_ptr,
    n_elements: tl.constexpr,
    scale_override: tl.constexpr,
    group_size: tl.constexpr,
    four_over_six: tl.constexpr,
    BLOCK_SIZE: tl.constexpr,
):        
    # load x
    pid = tl.program_id(0)
    start_idx = pid * BLOCK_SIZE
    offsets = start_idx + tl.arange(0, BLOCK_SIZE)
    mask = offsets < n_elements
    x_flat = tl.load(x_ptr + offsets, mask=mask, other=0.0)
    
    # group
    x_grouped = tl.reshape(x_flat, (BLOCK_SIZE // group_size, group_size))
    
    # amax
    scales_max = 256.00 if four_over_six else 448.00
    val_max = 6.0 / scale_override
    amax = tl.load(amax_ptr)
    
    s_dec_b_e4m3, s_dec = get_scales(x_grouped, amax, val_max, scales_max)
    x_scaled = x_grouped / (s_dec_b_e4m3 * s_dec)
    
    # quantize
    x_fp4 = rtn_fp4(x_scaled)
    x_dequantized = x_fp4 * (s_dec_b_e4m3 * s_dec)
    
    if not four_over_six:
        best_x_dequantized = x_dequantized
    else:
        alt_s_dec_b_e4m3 = get_alt_scales(x_grouped, val_max, s_dec)
        alt_x_scaled = x_grouped / (alt_s_dec_b_e4m3 * s_dec)
        
        alt_x_fp4 = rtn_fp4(alt_x_scaled)
        alt_x_dequantized = alt_x_fp4 * (alt_s_dec_b_e4m3 * s_dec)
        
        error_six = tl.sum((x_grouped - x_dequantized) * (x_grouped - x_dequantized), axis=-1, keep_dims=True)
        error_four = tl.sum((x_grouped - alt_x_dequantized) * (x_grouped - alt_x_dequantized), axis=-1, keep_dims=True)
        
        best_x_dequantized = tl.where(
            error_six <= error_four,
            x_dequantized,
            alt_x_dequantized,
        )
    
    x_dequantized_flat = tl.reshape(best_x_dequantized, (BLOCK_SIZE,))
    tl.store(output_ptr + offsets, x_dequantized_flat, mask=mask)


@torch.compiler.disable()
def rtn_1x16s_fp4_kernel_wrapper(
    x,
    scale_override: float,
    group_size: int,
    four_over_six: bool,
):
    x = x.contiguous()
    output = torch.empty_like(x)
    
    n_elements = x.numel()
    grid = lambda meta: (triton.cdiv(n_elements, meta["BLOCK_SIZE"]),)
    
    rtn_1x16s_fp4_kernel[grid](
        x_ptr=x,
        amax_ptr=x.abs().max(),
        output_ptr=output,
        n_elements=n_elements,
        scale_override=scale_override,
        group_size=group_size,
        four_over_six=four_over_six,
    )
    return output

In [3]:
@triton.autotune(
    configs=[
        triton.Config({"BLOCK_SIZE": 128 * 32}),
        triton.Config({"BLOCK_SIZE": 256 * 32}),
        triton.Config({"BLOCK_SIZE": 512 * 32}),
    ],
    key=[],
)
@triton.jit
def eden_1x16s_fp4_kernel(
    x_ptr,
    hadamard_matrix_ptr,
    current_amax_ptr,
    output_ptr,
    next_amax_ptr,
    n_elements: tl.constexpr,
    hadamard_dim: tl.constexpr,
    scale_override: tl.constexpr,
    group_size: tl.constexpr,
    seed: int,
    BLOCK_SIZE: tl.constexpr,
):    
    # load x
    pid = tl.program_id(0)
    start_idx = pid * BLOCK_SIZE
    offsets = start_idx + tl.arange(0, BLOCK_SIZE)
    mask = offsets < n_elements
    x_flat = tl.load(x_ptr + offsets, mask=mask, other=0.0)
    
    # hadamard transform
    offsets_hadamard = tl.arange(0, hadamard_dim * hadamard_dim)
    hadamard_matrix = tl.load(hadamard_matrix_ptr + offsets_hadamard).reshape(hadamard_dim, hadamard_dim) 
    x = tl.reshape(x_flat, (BLOCK_SIZE // hadamard_dim, hadamard_dim))
    x_had = tl.dot(x, hadamard_matrix) # not TN!, A @ B!
    
    # write amax for next iter
    tl.atomic_max(next_amax_ptr, tl.max(tl.abs(x_had)).to(tl.float32), sem="relaxed")
    
    # group
    x_grouped = tl.reshape(x_had, (BLOCK_SIZE // group_size, group_size))

    # amax
    scales_max = 255.99 # Not 448 because eden needs space to rescale up a bit sometimes after the correction
    val_max = 6.0 / scale_override
    amax = tl.load(current_amax_ptr)
    s_dec = tl.where(
        amax == 0.0,
        1.0,
        amax / scales_max / val_max,
    )
    
    # scale
    s_dec_b = tl.max(tl.abs(x_grouped), axis=-1, keep_dims=True) / val_max
    s_dec_b_e4m3 = (s_dec_b / s_dec).to(tl.float8e4nv).to(tl.float32)
    s_dec_b_e4m3 = tl.where(
        s_dec_b_e4m3 == 0,
        1.0,
        s_dec_b_e4m3,
    )
    x_scaled = x_grouped / (s_dec_b_e4m3 * s_dec)
    
    # quantize
    x_scaled_abs = tl.abs(x_scaled)
    x_scaled_sign = tl.where(
        x_scaled > 0,
        1,
        -1,
    )
    x_fp4 = tl.where(
        x_scaled_abs >= 5,
        6,
        tl.where(
            x_scaled_abs >= 3.5,
            4,
            tl.where(
                x_scaled_abs >= 2.5,
                3,
                tl.where(
                    x_scaled_abs >= 1.75,
                    2,
                    tl.where(
                        x_scaled_abs >= 1.25,
                        1.5,
                        tl.where(
                            x_scaled_abs >= 0.75,
                            1,
                            tl.where(
                                x_scaled_abs >= 0.25,
                                0.5,
                                0,
                            )
                        )
                    )
                )
            )
        )
    ) * x_scaled_sign
    
    # Calculate EDEN scale
    x_scaled = tl.reshape(x_scaled, (BLOCK_SIZE // hadamard_dim, hadamard_dim))
    x_fp4 = tl.reshape(x_fp4, (BLOCK_SIZE // hadamard_dim, hadamard_dim))
    
    num = tl.sum(x_scaled * x_scaled, axis=-1, keep_dims=True)
    denom = tl.sum(x_scaled * x_fp4, axis=-1, keep_dims=True)
    
    correction = tl.where(
        denom == 0.0,
        1.0,
        num / denom,
    )
    
    # Apply EDEN scale
    scales = tl.reshape(s_dec_b_e4m3, (BLOCK_SIZE // hadamard_dim, hadamard_dim // group_size))
    corrected_scales = tl.reshape(scales * correction, (BLOCK_SIZE // group_size, 1))
    
    bitscales = tl.cast(corrected_scales.to(tl.float8e4nv), tl.uint8, bitcast=True)
    prevscale = tl.cast((bitscales - 1), tl.float8e4nv, bitcast=True).to(tl.float32)
    currscale = tl.cast((bitscales), tl.float8e4nv, bitcast=True).to(tl.float32)
    nextscale = tl.cast((bitscales + 1), tl.float8e4nv, bitcast=True).to(tl.float32)
    
    up = tl.where(
        currscale > corrected_scales,
        currscale,
        nextscale,
    )
    down = tl.where(
        currscale > corrected_scales,
        prevscale,
        currscale,
    )
    
    prob_up = (corrected_scales - down) / (up - down)
    
    scale_start_idx = pid * (BLOCK_SIZE // group_size)
    scale_offsets = scale_start_idx + tl.arange(0, BLOCK_SIZE // group_size)
    sampled_prob = tl.rand(seed, scale_offsets).reshape(BLOCK_SIZE // group_size, 1)
    
    scales = tl.where(
        sampled_prob < prob_up,
        up,
        down,
    )
    scales = tl.reshape(scales, (BLOCK_SIZE // group_size, 1))
    x_fp4 = tl.reshape(x_fp4, (BLOCK_SIZE // group_size, group_size))
    
    # Reshape back to flat form for storage
    x_dequantized = x_fp4 * scales * s_dec
    x_dequantized_flat = tl.reshape(x_dequantized, (BLOCK_SIZE,))
    
    # store
    tl.store(output_ptr + offsets, x_dequantized_flat, mask=mask)

@torch.compiler.disable()
def eden_1x16s_fp4_kernel_wrapper(
    x: torch.Tensor,
    hadamard_matrix: torch.Tensor,
    scale_override: float,
    group_size: int,
    current_amax: torch.Tensor,
) -> [torch.Tensor, torch.Tensor]:
    hadamard_dim = hadamard_matrix.size(0)
    assert hadamard_matrix.size(1) == hadamard_dim
    assert (x.numel() // x.shape[-1]) % hadamard_dim == 0
    assert hadamard_dim % group_size == 0
    
    x = x.contiguous()
    hadamard_matrix = hadamard_matrix.T.contiguous() # .T.contiguous() + tl.dot -> TN
    output = torch.empty_like(x)
    seed = randint(0, 1000000)
    
    next_amax = torch.zeros_like(current_amax)
    
    n_elements = x.numel()
    grid = lambda meta: (triton.cdiv(n_elements, meta["BLOCK_SIZE"]),)
    
    eden_1x16s_fp4_kernel[grid](
        x_ptr=x,
        hadamard_matrix_ptr=hadamard_matrix,
        current_amax_ptr=current_amax,
        output_ptr=output,
        next_amax_ptr=next_amax,
        n_elements=n_elements,
        hadamard_dim=hadamard_dim,
        scale_override=scale_override,
        group_size=group_size,
        seed=seed,
    )
    return output, next_amax


In [4]:
from scipy.linalg import hadamard

def get_hadamard_matrix(group_size: int, dtype: torch.dtype, device: torch.device):
    return torch.tensor(
        hadamard(group_size) * group_size**-0.5,
        dtype=dtype,
        device=device,
        requires_grad=False,
    )
    
def rerotate_hadamard(hadamard_matrix):
    signs = torch.diag(
        torch.randint(
            0, 2, (hadamard_matrix.size(0),),
            device=hadamard_matrix.device,
            dtype=hadamard_matrix.dtype
        ) * 2 - 1
    )
    return hadamard_matrix @ signs # NOTE: rerotate along last dim, inner dim for TN GEMM

In [5]:
from tqdm.auto import trange, tqdm

M = 1024
N = 1024
K = 1024
HADAMARD_DIM = 128

A = torch.randn((M, K), device='cuda')
B = torch.randn((N, K), device='cuda')
ht = get_hadamard_matrix(32, A.dtype, A.device)

with torch.no_grad():
    for acc_steps in tqdm([1, 4, 16, 64, 256, 1024], desc="Iterating steps"):
        accumulator = torch.zeros_like(A @ B.T)
        for i in trange(acc_steps, leave=False):
            ht = rerotate_hadamard(ht)

            A_amax_buffer = (A.view(-1, ht.size(0)) @ ht.T).abs().max()
            Aq, A_amax_buffer = eden_1x16s_fp4_kernel_wrapper(
                A,
                ht,
                1.0,
                16,
                current_amax=A_amax_buffer,
            )
            
            B_amax_buffer = (A.view(-1, ht.size(0)) @ ht.T).abs().max()
            Bq, B_amax_buffer = eden_1x16s_fp4_kernel_wrapper(
                B,
                ht,
                1.0,
                16,
                current_amax=B_amax_buffer,
            )
            
            accumulator += Aq @ Bq.T
        accumulator /= acc_steps
        
        quad_err = (accumulator - A @ B.T).pow(2).mean() / (A @ B.T).pow(2).mean()
        eff_bitwidth = (-torch.log2(quad_err) / 2).item()
        print(f"{acc_steps}: {eff_bitwidth:.2f} bits")
        
# NEED TO GROW BY ~1 bit per 4x samples
# v

/nfs/scistore19/alistgrp/apanfero/micromamba/envs/llmb/lib/python3.10/site-packages/torch/cuda/__init__.py:799: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Iterating steps:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

1: 2.84 bits


  0%|          | 0/4 [00:00<?, ?it/s]

...

64: 5.83 bits


  0%|          | 0/256 [00:00<?, ?it/s]

256: 6.81 bits


  0%|          | 0/1024 [00:00<?, ?it/s]

1024: 7.75 bits


## Quartet_II linear

In [6]:
class AmaxStorage:
    def __init__(self):
        self.e_ht_amax = None
        self.weght_tht_amax = None
        self.e_tht_amax = None
        self.input_tht_amax = None
        
    def __repr__(self) -> str:
        fields = [
            ("e_ht_amax", self.e_ht_amax), 
            ("weght_tht_amax", self.weght_tht_amax), 
            ("e_tht_amax", self.e_tht_amax), 
            ("input_tht_amax", self.input_tht_amax)
        ]
        field_strs = []
        for name, val in fields:
            if val is not None:
                try:
                    v = val.item()
                except Exception:
                    v = val
                field_strs.append(f"{name}: {v:.3e}")
            else:
                field_strs.append(f"{name}: None")
        return "<AmaxStorage " + ", ".join(field_strs) + ">"
        

class Quartet_II_fn(torch.autograd.Function):
    group_size = 16
    forward_scale_override = 1.0
    backward_scale_override = (17 / 16) * 0.93
    hadamard_matrix = get_hadamard_matrix(128, device="cuda", dtype=torch.float32)

    @torch.compile(dynamic=False)
    @staticmethod
    def forward(ctx, input, weight, amax_storage: AmaxStorage, delayed_amax: bool, disable_backward_quant: bool, four_over_six: bool):
        ctx.batch = input.shape[0]
        ctx.seq = input.shape[1]
        ctx.in_dim = weight.shape[1]
        ctx.out_dim = weight.shape[0]
        ctx.delayed_amax = delayed_amax
        ctx.amax_storage = amax_storage
        ctx.disable_backward_quant = disable_backward_quant
        
        input_fp4 = rtn_1x16s_fp4_kernel_wrapper(input, scale_override=Quartet_II_fn.forward_scale_override, group_size=Quartet_II_fn.group_size, four_over_six=four_over_six)
        weight_fp4 = rtn_1x16s_fp4_kernel_wrapper(weight.to(input.dtype), scale_override=Quartet_II_fn.forward_scale_override, group_size=Quartet_II_fn.group_size, four_over_six=four_over_six)

        ctx.save_for_backward(input_fp4, weight_fp4)
        return F.linear(input_fp4, weight_fp4)

    @torch.compile(dynamic=False)
    @staticmethod
    def backward(ctx, grad_output):
        # Load ctx and reshape
        input_fp4, weight_fp4 = ctx.saved_tensors
        
        input_fp4 = input_fp4.reshape(ctx.batch * ctx.seq, ctx.in_dim)
        grad_output = grad_output.reshape(ctx.batch * ctx.seq, ctx.out_dim)
        
        # Re-randomize the rotation
        Quartet_II_fn.hadamard_matrix = Quartet_II_fn.hadamard_matrix.to(grad_output.dtype)
        Quartet_II_fn.hadamard_matrix = rerotate_hadamard(Quartet_II_fn.hadamard_matrix)
        
        # No backward quant if flag
        if ctx.disable_backward_quant:
            grad_input = F.linear(
                grad_output,
                weight_fp4.T,
                None,
            ).view(ctx.batch, ctx.seq, ctx.in_dim)
            
            grad_weight = F.linear(
                grad_output.T,
                input_fp4.T,
                None,
            )
            return grad_input, grad_weight, None, None, None, None
        
        # EW
        if ctx.amax_storage.e_ht_amax is None or not ctx.delayed_amax:
            ctx.amax_storage.e_ht_amax = (grad_output.reshape(-1, Quartet_II_fn.hadamard_matrix.size(0)) @ Quartet_II_fn.hadamard_matrix.T).abs().max().float()
        e_ht_fp4, ctx.amax_storage.e_ht_amax = eden_1x16s_fp4_kernel_wrapper(grad_output, Quartet_II_fn.hadamard_matrix, Quartet_II_fn.backward_scale_override, Quartet_II_fn.group_size, ctx.amax_storage.e_ht_amax)
        
        if ctx.amax_storage.weght_tht_amax is None or not ctx.delayed_amax:
            ctx.amax_storage.weght_tht_amax = (weight_fp4.T.reshape(-1, Quartet_II_fn.hadamard_matrix.size(0)) @ Quartet_II_fn.hadamard_matrix.T).abs().max().float()
        weight_tht_fp4, ctx.amax_storage.weght_tht_amax = eden_1x16s_fp4_kernel_wrapper(weight_fp4.T, Quartet_II_fn.hadamard_matrix, Quartet_II_fn.backward_scale_override, Quartet_II_fn.group_size, ctx.amax_storage.weght_tht_amax)
        
        grad_input = F.linear(
            e_ht_fp4,
            weight_tht_fp4,
            None,
        ).view(ctx.batch, ctx.seq, ctx.in_dim)

        # EtX
        if ctx.amax_storage.e_tht_amax is None or not ctx.delayed_amax:
            ctx.amax_storage.e_tht_amax = (grad_output.T.reshape(-1, Quartet_II_fn.hadamard_matrix.size(0)) @ Quartet_II_fn.hadamard_matrix.T).abs().max().float()
        e_tht_fp4, ctx.amax_storage.e_tht_amax = eden_1x16s_fp4_kernel_wrapper(grad_output.T, Quartet_II_fn.hadamard_matrix, Quartet_II_fn.backward_scale_override, Quartet_II_fn.group_size, ctx.amax_storage.e_tht_amax)
        
        if ctx.amax_storage.input_tht_amax is None or not ctx.delayed_amax:
            ctx.amax_storage.input_tht_amax = (input_fp4.T.reshape(-1, Quartet_II_fn.hadamard_matrix.size(0)) @ Quartet_II_fn.hadamard_matrix.T).abs().max().float()
        input_tht_fp4, ctx.amax_storage.input_tht_amax = eden_1x16s_fp4_kernel_wrapper(input_fp4.T, Quartet_II_fn.hadamard_matrix, Quartet_II_fn.backward_scale_override, Quartet_II_fn.group_size, ctx.amax_storage.input_tht_amax)
        
        grad_weight = F.linear(
            e_tht_fp4,
            input_tht_fp4,
            None,
        )
        
        return grad_input, grad_weight, None, None, None, None

In [7]:
class Quartet_II_linear(torch.nn.Linear):
    def __init__(self, *args, delayed_amax=False, disable_backward_quant=False, four_over_six=True, **kwargs):
        super().__init__(*args, **kwargs)
        self.delayed_amax = delayed_amax
        self.disable_backward_quant = disable_backward_quant
        self.four_over_six = four_over_six
        self.amax_storage = AmaxStorage()
        
    
    def forward(self, x, disable_backward_quant=False):
        return Quartet_II_fn.apply(x, self.weight, self.amax_storage, self.delayed_amax, self.disable_backward_quant, self.four_over_six)
    

In [8]:
BATCH = 4
SEQ = 128
HID = 256
DELAYED_AMAX = True

INPUT = torch.randn((BATCH, SEQ, HID), device='cuda')
TARGET = torch.randn((BATCH, SEQ, 1), device='cuda')

W1 = Quartet_II_linear(HID, HID, device='cuda', delayed_amax=DELAYED_AMAX)
W2 = Quartet_II_linear(HID, HID, device='cuda', delayed_amax=DELAYED_AMAX)
W3 = Quartet_II_linear(HID, HID, device='cuda', delayed_amax=DELAYED_AMAX)

with torch.no_grad():
    W1.weight /= (HID**0.5 * W1.weight.std())
    W2.weight /= (HID**0.5 * W2.weight.std())
    W3.weight /= (HID**0.5 * W3.weight.std())

head = torch.randn(HID, 1, device='cuda')

In [9]:
W1.weight.grad = None
W2.weight.grad = None
W3.weight.grad = None

W1.disable_backward_quant = True
W2.disable_backward_quant = True
W3.disable_backward_quant = True

hid = W1(INPUT)
hid = torch.nn.functional.relu(hid)
hid = W2(hid)
hid = torch.nn.functional.relu(hid)
hid = W3(hid)
loss = (hid @ head - TARGET).pow(2).mean()
loss.backward()

w1_ref_grad = W1.weight.grad.clone().detach()
w2_ref_grad = W2.weight.grad.clone().detach()
w3_ref_grad = W3.weight.grad.clone().detach()

/nfs/scistore19/alistgrp/apanfero/micromamba/envs/llmb/lib/python3.10/site-packages/torch/cuda/__init__.py:799: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [10]:
W1.disable_backward_quant = False
W2.disable_backward_quant = False
W3.disable_backward_quant = False

hid = W1(INPUT)
hid = torch.nn.functional.relu(hid)
hid = W2(hid)
hid = torch.nn.functional.relu(hid)
hid = W3(hid)
loss = (hid @ head - TARGET).pow(2).mean()

for acc_steps in tqdm([1, 4, 16, 64, 256, 1024, 4096]):
    print(f"{acc_steps=}:")
    W1.weight.grad = None
    W2.weight.grad = None
    W3.weight.grad = None
    for _ in trange(acc_steps, leave=False):
        loss.backward(retain_graph=True)
    with torch.no_grad():
        W1.weight.grad /= acc_steps
        W2.weight.grad /= acc_steps
        W3.weight.grad /= acc_steps
    
        quad_err = (W1.weight.grad - w1_ref_grad).pow(2).mean() / w1_ref_grad.pow(2).mean()
        eff_bitwidth = (-torch.log2(quad_err) / 2).item()
        cosine = (W1.weight.grad.flatten() @ w1_ref_grad.flatten()) / (w1_ref_grad.flatten() @ w1_ref_grad.flatten())
        print(f"\tW1 grad err: {eff_bitwidth:.2f} bits, {cosine:.3f} cosine")
        
        quad_err = (W2.weight.grad - w2_ref_grad).pow(2).mean() / w2_ref_grad.pow(2).mean()
        eff_bitwidth = (-torch.log2(quad_err) / 2).item()
        cosine = (W2.weight.grad.flatten() @ w2_ref_grad.flatten()) / (w2_ref_grad.flatten() @ w2_ref_grad.flatten())
        print(f"\tW2 grad err: {eff_bitwidth:.2f} bits, {cosine:.3f} cosine")
        
        quad_err = (W3.weight.grad - w3_ref_grad).pow(2).mean() / w3_ref_grad.pow(2).mean()
        eff_bitwidth = (-torch.log2(quad_err) / 2).item()
        cosine = (W3.weight.grad.flatten() @ w3_ref_grad.flatten()) / (w3_ref_grad.flatten() @ w3_ref_grad.flatten())
        print(f"\tW3 grad err: {eff_bitwidth:.2f} bits, {cosine:.3f} cosine")


# NEED TO GROW BY ~1 bit per 4x samples
# v

  0%|          | 0/7 [00:00<?, ?it/s]

acc_steps=1:


  0%|          | 0/1 [00:00<?, ?it/s]

/nfs/scistore19/alistgrp/apanfero/micromamba/envs/llmb/lib/python3.10/site-packages/torch/_dynamo/variables/functions.py:1391: UserWarning: Dynamo does not know how to trace the builtin `<unknown module>.Tensor.reshape.` This function is either a Python builtin (e.g. _warnings.warn) or a third-party C/C++ Python extension (perhaps created with pybind).
If it is a Python builtin, please file an issue on GitHub so the PyTorch team can add support for it and see the next case for a workaround.
If it is a third-party C/C++ Python extension, please either wrap it into a PyTorch-understood custom operator (see https://pytorch.org/tutorials/advanced/custom_ops_landing_page.html for more details) or, if it is traceable, use `torch.compiler.allow_in_graph`.
  torch._dynamo.utils.warn_once(explanation + "\n" + "\n".join(hints))


	W1 grad err: 2.21 bits, 0.996 cosine
	W2 grad err: 2.80 bits, 0.999 cosine
	W3 grad err: 4.06 bits, 1.002 cosine
acc_steps=4:


...

	W1 grad err: 6.05 bits, 0.994 cosine
	W2 grad err: 6.67 bits, 0.997 cosine
	W3 grad err: 7.77 bits, 0.997 cosine
acc_steps=1024:


  0%|          | 0/1024 [00:00<?, ?it/s]

	W1 grad err: 6.79 bits, 0.994 cosine
	W2 grad err: 7.36 bits, 0.996 cosine
	W3 grad err: 8.63 bits, 0.999 cosine
acc_steps=4096:


  0%|          | 0/4096 [00:00<?, ?it/s]

	W1 grad err: 7.22 bits, 0.994 cosine
	W2 grad err: 7.73 bits, 0.996 cosine
	W3 grad err: 8.44 bits, 0.997 cosine


In [11]:
def bench_shape(
    in_dim, out_dim, batch_size, seq_len,
    weight_dtype=torch.float32, act_dtype=torch.bfloat16, device='cuda',
    warmup=10, rep=100,
    compile=True, compile_kwargs=None,
):
    if compile_kwargs is None:
        compile_kwargs = {'dynamic': False, 'mode': 'reduce-overhead', 'fullgraph': False}
    
    x = torch.randn(batch_size, seq_len, in_dim, device=device, dtype=act_dtype)
    
    linear = Quartet_II_linear(in_dim, out_dim, four_over_six=True, device=device, dtype=weight_dtype)
    if compile:
        linear = torch.compile(linear, **compile_kwargs)
    
    # Forward
    torch.set_grad_enabled(False)
    
    ms = triton.testing.do_bench(
        lambda: linear(x), warmup=warmup, rep=rep,
    )
    forward_time = ms

    # Forward+Backward
    grad = torch.randn_like(linear(x))
    torch.set_grad_enabled(True)    
    
    def forward_backward(x, grad):
        output = linear(x)
        output.backward(grad)
            
    if compile:
        compiled_forward_backward = torch.compile(forward_backward, **compile_kwargs)
    
    ms = triton.testing.do_bench(
        lambda: compiled_forward_backward(x, grad), warmup=warmup, rep=rep,
    )
    total_time = ms

    return {
        "forward_ms": forward_time,
        "total_ms": total_time,
    }

In [12]:
from tqdm.auto import tqdm, trange

BATCH_SIZE=8
SEQ_LEN = 2048

shapes = {
    # Q K V Down Up Gate Down
    # "100M": [(1024 * 3, 1024), (1024, 1024), (2816 * 2, 1024), (1024, 2816)],
    # "800M": [(2048 * 3, 2048), (2048, 2048), (5632 * 2, 2048), (2048, 5632)],
    "3B": [(3072 * 3, 3072), (3072, 3072), (8192 * 2, 3072), (3072, 8192)],
    "7B": [(4096 * 3, 4096), (4096, 4096), (11008 * 2, 4096), (4096, 11008)],
    "22B": [(6144 * 3, 6144), (6144, 6144), (16384 * 2, 6144), (6144, 16384)],
    # "52B": [(8192 * 3, 8192), (8192, 8192), (22016 * 2, 8192), (8192, 22016)],
}

shape_to_result = {}

for size, model_shapes in tqdm(shapes.items(), desc="Iterating model sizes"):
    for shape in tqdm(model_shapes, desc="Iterating shapes", leave=False):
        if shape not in shape_to_result:
            result = bench_shape(
                shape[1], shape[0], BATCH_SIZE, SEQ_LEN,
            )
            shape_to_result[shape] = result
        


Iterating model sizes:   0%|          | 0/3 [00:00<?, ?it/s]

Iterating shapes:   0%|          | 0/4 [00:00<?, ?it/s]

W0121 18:16:56.739000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [10/8] torch._dynamo hit config.recompile_limit (8)
W0121 18:16:56.739000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [10/8]    function: 'torch_dynamo_resume_in_backward_at_81' (/tmp/ipykernel_135523/194258891.py:81)
W0121 18:16:56.739000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [10/8]    last reason: 10/0: Cache line invalidated because L['___stack0'] got deallocated
W0121 18:16:56.739000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [10/8] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W0121 18:16:56.739000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [10/8] To diagnose recompilation issues, see https://pytorch.org/docs/main/torch.compiler_troubleshooting.html.
W0121 18:17:01.013000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [9/8] torch._dynamo hit config.recompile_limit (8)
W0121 18:17:01.013000 135523 site-packages/torch/_

Iterating shapes:   0%|          | 0/4 [00:00<?, ?it/s]

W0121 18:17:06.342000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [16/8] torch._dynamo hit config.recompile_limit (8)
W0121 18:17:06.342000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [16/8]    function: 'forward' (/tmp/ipykernel_135523/1317231249.py:10)
W0121 18:17:06.342000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [16/8]    last reason: 16/7: GLOBAL_STATE changed: grad_mode 
W0121 18:17:06.342000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [16/8] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W0121 18:17:06.342000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [16/8] To diagnose recompilation issues, see https://pytorch.org/docs/main/torch.compiler_troubleshooting.html.
W0121 18:17:06.345000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [1/8] torch._dynamo hit config.recompile_limit (8)
W0121 18:17:06.345000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [1/8]    function: 'forward' 

Iterating shapes:   0%|          | 0/4 [00:00<?, ?it/s]

W0121 18:17:27.162000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [18/8] torch._dynamo hit config.recompile_limit (8)
W0121 18:17:27.162000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [18/8]    function: 'torch_dynamo_resume_in_forward_backward_at_29' (/tmp/ipykernel_135523/3247581210.py:29)
W0121 18:17:27.162000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [18/8]    last reason: 18/7: tensor 'grad' size mismatch at index 2. expected 6144, actual 32768
W0121 18:17:27.162000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [18/8] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W0121 18:17:27.162000 135523 site-packages/torch/_dynamo/convert_frame.py:987] [18/8] To diagnose recompilation issues, see https://pytorch.org/docs/main/torch.compiler_troubleshooting.html.


In [13]:
for size, model_shapes in shapes.items():
    forward_latency = sum(shape_to_result[shape]['forward_ms'] for shape in model_shapes)
    total_latency = sum(shape_to_result[shape]['total_ms'] for shape in model_shapes)
    
    print(f"{size}: {forward_latency}ms forward, {total_latency}ms foward+backward")

3B: 20.671732938976817ms forward, 107.73571999867757ms foward+backward
7B: 36.1965014255408ms forward, 170.70031929016113ms foward+backward
22B: 79.49745286794808ms forward, 650.2594998677571ms foward+backward
